In [ ]:
from pyspark.sql.functions import col, round
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

Create and mount data

In [ ]:
# Example with secrets management for sensitive information
client_id = "{generate your own while creating new app registration}"
client_secret = "{generate your own while creating new app registration}"
tenant_id = "{generate your own while creating new app registration}"

# Define storage account configurations
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": client_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

# Define source and mount point
source = "abfss://tokyoolympicdatasets@tokyoolympicdatasagarmoy.dfs.core.windows.net"
mount_point = "/mnt/tokyoolympicnew"

try:
    # Mount the storage account
    dbutils.fs.mount(
        source=source,
        mount_point=mount_point,
        extra_configs=configs
    )
    print(f"Mounted {source} to {mount_point} successfully.")
except Exception as e:
    print(f"Error mounting storage: {e}")


Mounted abfss://tokyoolympicdatasets@tokyoolympicdatasagarmoy.dfs.core.windows.net to /mnt/tokyoolympicnew successfully.


In [ ]:
%fs
ls "/mnt/tokyoolympicnew"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympicnew/gold-data/,gold-data/,0,1720339719000
dbfs:/mnt/tokyoolympicnew/raw-data/,raw-data/,0,1720339709000


In [ ]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [ ]:
athletes = spark.read.format("csv").option("header","true").load("/mnt/tokyoolympicnew/raw-data/raw_athletes.csv")

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
# lets load all the files
coaches = spark.read.format("csv").option("header","true").load("/mnt/tokyoolympicnew/raw-data/raw_coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").load("/mnt/tokyoolympicnew/raw-data/raw_entriesgender.csv")
medals = spark.read.format("csv").option("header","true").load("/mnt/tokyoolympicnew/raw-data/raw_medals.csv")
teams = spark.read.format("csv").option("header","true").load("/mnt/tokyoolympicnew/raw-data/raw_teams.csv")

Basic Transformation

In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [ ]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [ ]:
# changing the datatype of the "Female", "Male", "Total" from string to integer

entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType())).withColumn("Male",col("Male").cast(IntegerType())).withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [ ]:
# changing the numeric columns datatype from string to integer or we can add (.option("inferSchema","true")) while loading the data to automatically identify and add correct datatype

medals = medals.withColumn("Gold",col("Gold").cast(IntegerType())).withColumn("Silver",col("Silver").cast(IntegerType())).withColumn("Bronze",col("Bronze").cast(IntegerType())).withColumn("Total",col("Total").cast(IntegerType())).withColumn("Rank by Total",col("Rank by Total").cast(IntegerType()))

In [ ]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [ ]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', round(entriesgender['Female'] / entriesgender['Total'], 2)
).withColumn(
    'Avg_Male', round(entriesgender['Male'] / entriesgender['Total'], 2)
)
average_entries_by_gender.show()

+--------------------+------+----+-----+----------+--------+
|          Discipline|Female|Male|Total|Avg_Female|Avg_Male|
+--------------------+------+----+-----+----------+--------+
|      3x3 Basketball|    32|  32|   64|       0.5|     0.5|
|             Archery|    64|  64|  128|       0.5|     0.5|
| Artistic Gymnastics|    98|  98|  196|       0.5|     0.5|
|   Artistic Swimming|   105|   0|  105|       1.0|     0.0|
|           Athletics|   969|1072| 2041|      0.47|    0.53|
|           Badminton|    86|  87|  173|       0.5|     0.5|
|   Baseball/Softball|    90| 144|  234|      0.38|    0.62|
|          Basketball|   144| 144|  288|       0.5|     0.5|
|    Beach Volleyball|    48|  48|   96|       0.5|     0.5|
|              Boxing|   102| 187|  289|      0.35|    0.65|
|        Canoe Slalom|    41|  41|   82|       0.5|     0.5|
|        Canoe Sprint|   123| 126|  249|      0.49|    0.51|
|Cycling BMX Frees...|    10|   9|   19|      0.53|    0.47|
|  Cycling BMX Racing|  

Write transformed files to storage

In [ ]:
# to overwrite --> .mode("overwrite")
# to create n number of partition of the written file --> .repartition(n)

athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympicnew/gold-data/athletes")


In [ ]:
# write all other files

coaches.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympicnew/gold-data/coaches")

entriesgender.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympicnew/gold-data/entriesgender")

medals.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympicnew/gold-data/medals")

teams.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympicnew/gold-data/teams")

average_entries_by_gender.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympicnew/gold-data/average_entries_by_gender")

END